## Загрузка модели

In [ ]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 1.8MB 7.9MB/s 
     |████████████████████████████████| 890kB 54.0MB/s 
     |████████████████████████████████| 3.2MB 54.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=fae258af31a5bee372670787858c6c497db548acd5c30c12ef61026084e638e1
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 163kB 6.7MB/s 
     |████████████████████████████████| 245kB 32.2MB/s 
     |████████████████████████████████| 20.7MB 1.4MB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model_begin = BertModel.from_pretrained("DeepPavlov/rubert-base-cased-conversational")

import torch
from torch import nn

## Тесты(функции)

In [ ]:
"""
Класс, который берёт слово и выдаёт эмбед этого слова.
Нужен для check_similarity
"""
class EmbedModel:
    """
    @param word_to_token Объект, преобразовывающий слово в токены (BertTokenizer)
    @param model Объект, преобразовывающий токены в эмбед (BertModel.embeddings.word_embeddings)
    Если подаётся BertModel, конвертирует его в BertModel.embeddings.word_embeddings
    """
    def __init__(self, word_to_token, token_to_embed):
        self.word_to_token = word_to_token
        if isinstance(token_to_embed, BertModel):
            self.token_to_embed = token_to_embed.embeddings.word_embeddings
        else:
            self.token_to_embed = token_to_embed

    def __call__(self, word):
        tokens = torch.Tensor(self.word_to_token.encode(word, add_special_tokens=False)).long()
        out = self.token_to_embed(tokens)
        return out.mean(dim=0)

baseline_model = EmbedModel(tokenizer, model_begin)

"""
Обрабатывает тест.
Также, если silent=False, выводит расстояния между каждой парой слов. 
@param model Модель, которая берёт слово и выдаёт эмбед этого слова
@param similar_word1 Первое слово, похожее на второе по смыслу
@param similar_word2 Второе слово, похожее на первое по смыслу
@param different_word Третье слово, отличающиеся от двух других по смыслу
@param silent Отключить вывод расстояний
@returns True если эмбеды первых двух слов ближе, чем к третьему; False иначе 
"""
def check_similarity(model, similar_word1, similar_word2, different_word, silent=False):
    v1, v2, v3 = model(similar_word1), model(similar_word2), model(different_word)
    d12 = float(nn.CosineSimilarity(dim=0)(v1, v2))
    d13 = float(nn.CosineSimilarity(dim=0)(v1, v3))
    d23 = float(nn.CosineSimilarity(dim=0)(v2, v3))
    if not silent:
        print('Distance between similar words:', d12)
        print('Distance between first and third words:', d13)
        print('Distance between second and third words:', d23)
    return (d12 > d13) and (d12 > d23)
  
print(check_similarity(baseline_model, "вошёл", "зашёл", "красивый"))
print(check_similarity(baseline_model, "вошёл", "пришёл", "красивый"))

Distance between similar words: 0.576931357383728
Distance between first and third words: 0.5771180391311646
Distance between second and third words: 0.5649428367614746
False
Distance between similar words: 0.5449704527854919
Distance between first and third words: 0.5771180391311646
Distance between second and third words: 0.58338463306427
False


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
model_test = BertModel.from_pretrained('/content/drive/MyDrive/chatbot_project/models/saved_model_tuned/')

In [ ]:
print(check_similarity(EmbedModel(tokenizer, model_test), "сидеть", "сидя", "котики"))
print(check_similarity(EmbedModel(tokenizer, model_test), "первый", "второй", "синий"))
print(check_similarity(EmbedModel(tokenizer, model_test), "животное", "корова", "выпендриваться"))
print(check_similarity(EmbedModel(tokenizer, model_test), "а", "о", "союзы"))
print(check_similarity(EmbedModel(tokenizer, model_test), "король", "королева", "тридцатидевятимиллиметровый"))

Distance between similar words: 0.5600447058677673
Distance between first and third words: 0.5069496631622314
Distance between second and third words: 0.650918185710907
False
Distance between similar words: 0.6366925239562988
Distance between first and third words: 0.610293984413147
Distance between second and third words: 0.6655640006065369
False
Distance between similar words: 0.5550908446311951
Distance between first and third words: 0.6973404884338379
Distance between second and third words: 0.6581167578697205
False
Distance between similar words: 0.6205568909645081
Distance between first and third words: 0.6041322350502014
Distance between second and third words: 0.6441978812217712
False
Distance between similar words: 0.7035229206085205
Distance between first and third words: 0.6949911117553711
Distance between second and third words: 0.7271926403045654
False


In [ ]:
import torch
cuda = torch.device('cuda:0')

def make_sentence(tokens):
  s = ''
  for token in tokens:
    if ('##' in token):
      s += token.replace('##', '')
    else:
      s += ' ' + token
  return s

def predict(target="Вася покачал головой", pos=2, num_out=5):
  inputs = tokenizer(target, return_tensors="pt").to(cuda)
  print(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0]))
  print(f"Pos = {pos}")
  print()
  outputs = model.forward(**inputs, output_hidden_states=True)
  predictions = outputs.logits
  predicted_index = torch.argsort(predictions[0, pos])
  splited = target.split()
  for i in range(num_out):
      ind = predicted_index[-1-i]
      res = inputs['input_ids'][0]
      tokens_cnt = len(res)
      res[pos] = ind.item()
      tokens = tokenizer.convert_ids_to_tokens(res)
      predicted_token = tokenizer.convert_ids_to_tokens([ind])[0]
      print(f"Index = {ind.item()}")
      print(f"Value = {predictions[0, pos, ind].item()}")
      print(f"Token = {predicted_token}")
      s = make_sentence(tokens)
      print(s)
      print()

## Загрузка датасета



In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

url_facts4 = 'https://raw.githubusercontent.com/Koziev/NLP_Datasets/master/Samples/facts4.txt'
url_facts5 = 'https://raw.githubusercontent.com/Koziev/NLP_Datasets/master/Samples/facts5.txt'
url_facts6 = 'https://raw.githubusercontent.com/Koziev/NLP_Datasets/master/Samples/facts6.txt'
url_facts7_1s = 'https://raw.githubusercontent.com/Koziev/NLP_Datasets/master/Samples/facts7_1s.txt'
url_facts7_2s = 'https://raw.githubusercontent.com/Koziev/NLP_Datasets/master/Samples/facts7_2s.txt'
url_big_data = '/content/drive/MyDrive/chatbot_project/data/data_concat.txt'
url_lit = '/content/drive/MyDrive/chatbot_project/literature.txt'

Mounted at /content/drive


In [ ]:
data_facts4 = pd.read_csv(url_facts4, sep="\t", header=None)
data_facts5 = pd.read_csv(url_facts5, sep="\t", header=None)
data_facts6 = pd.read_csv(url_facts6, sep="\t", header=None)
data_facts7_1s = pd.read_csv(url_facts7_1s, sep="\t", header=None)
data_facts7_2s = pd.read_csv(url_facts7_2s, sep="\t", header=None)
data_lit = pd.read_csv(url_lit, sep="\t", header=None)
# data_big = pd.read_csv(url_big_data, sep="\t", header=None)

In [ ]:
data_big = pd.concat([data_facts4, data_facts5, data_facts6, data_facts7_1s, data_facts7_2s, data_lit])

In [ ]:
data_big

,0
0,Дик покачал головой
1,Яр покачал головой
2,Кат покачал головой
3,Вася покачал головой
4,Рон покачал головой
...,...
1825446,"Они были мертвы, зародыши в"
1825447,них были прикончены.
1825448,Необозримые пространства земли еще долго гнил...
1825449,"крокодилов и змей, вызванных к жизни таин..."


In [ ]:
# порежем датасет, иначе очень долго обучаться и слишком много однотипных вариантов
data = data_big
msk = np.random.rand(len(data)) < 0.2
data = data[msk]

In [ ]:
train_part = 0.9

msk = np.random.rand(len(data)) < train_part
data_train = data[msk] 
data_test = data[~msk]

print("train len =", len(data_train), " test len =", len(data_test))

train len = 396693  test len = 44533


In [ ]:
from datasets import Dataset

dataset_train = Dataset.from_pandas(data_train)
dataset_eval = Dataset.from_pandas(data_test)

dataset_train = dataset_train.shuffle()
dataset_eval = dataset_eval.shuffle()

In [ ]:
dataset_train

Dataset({
    features: ['0', '__index_level_0__'],
    num_rows: 396693
})

In [ ]:
def tokenize_function(examples):
    res = tokenizer(examples["0"])
    if len(res['input_ids']) > 30:
        for name in res:
            res[name] = res[name][:30] + [res[name][-1]]
    return res

In [ ]:
tokenized_dataset_train = dataset_train.map(tokenize_function, batched=False, num_proc=4, remove_columns=["0", "__index_level_0__"])
tokenized_dataset_eval = dataset_eval.map(tokenize_function, batched=False, num_proc=4, remove_columns=["0", "__index_level_0__"])

In [ ]:
tokenized_dataset_train

Dataset({
    features: ['attention_mask', 'input_ids', 'token_type_ids'],
    num_rows: 396693
})

## Дообучение

### перезагрузка модели 

In [ ]:
model_begin.save_pretrained('./saved_model')

In [ ]:
from transformers import AutoModelForMaskedLM
# model = AutoModelForMaskedLM.from_pretrained('./saved_model')

In [ ]:
model = AutoModelForMaskedLM.from_pretrained('/content/drive/MyDrive/chatbot_project/models/tuned0')

### заморозка



#### замораживаем параметры

In [ ]:
for param in model.base_model.parameters():
     param.requires_grad = False

In [ ]:
model.cls.predictions.decoder.weight = torch.nn.parameter.Parameter(\
                          model.cls.predictions.decoder.weight.clone().detach()\
                                                                    )

In [ ]:
def check(model):
    for param in model.named_parameters():
        print('1' if param[1].requires_grad else '0', param[0])

#### custom trainer

Всё тут написанное ради кастомного разбиения на батчи, чтобы не было [PAD] вместо половины токенов.

Код взять с гитхаба huggingface и модифицирован под задачу.
Основная функция get_indices, а остально обёртки для совместимости с библиотекой.

In [ ]:
from transformers import Trainer, TrainingArguments
from torch.utils.data.dataloader import DataLoader

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [ ]:
# from https://github.com/huggingface/transformers/blob/master/src/transformers/trainer_pt_utils.py
# тут лежат подобные функции от huggingface
# их ещё не добавили в библиотеку и вообще там странный код с другим предназначением

import math
from torch.utils.data.distributed import DistributedSampler
from typing import Iterator, List, Optional, Union

def get_indices(lengths, batch_size, generator=None):
    # применяем случайную перестановку к датасету и сортируем по длине
    indices = torch.randperm(len(lengths), generator=generator)
    len_sorted = list(sorted(indices.tolist(), key=lambda i: lengths[i], reverse=True)) 

    # делим на батчи (почти во всех батчах длина одинаковая)
    batches = [len_sorted[i : i + batch_size] for i in range(0, len(lengths), batch_size)]
    batches_cnt = len(batches)
  
    # применяем случайную перестановку к батчам
    # в конец докидывается последний батч, так как он может быть меньше batch_size
    batch_indices = torch.randperm(batches_cnt - 1, generator=generator)
    rand_batches = [batches[batch_ind] for batch_ind in batch_indices]
    res = sum(rand_batches, []) + batches[batches_cnt - 1]
    return res 

class DistributedLengthSortedSampler(DistributedSampler):
    # Copied and adapted from PyTorch DistributedSampler.
    # https://github.com/pytorch/pytorch/blob/master/torch/utils/data/distributed.py

    def __init__(
        self,
        dataset: Dataset,
        batch_size: int,
        num_replicas: Optional[int] = None,
        rank: Optional[int] = None,
        seed: int = 0,
        drop_last: bool = False,
        lengths: Optional[List[int]] = None,
    ):
        if num_replicas is None:
            if not dist.is_available():
                raise RuntimeError("Requires distributed package to be available")
            num_replicas = dist.get_world_size()
        if rank is None:
            if not dist.is_available():
                raise RuntimeError("Requires distributed package to be available")
            rank = dist.get_rank()
        self.dataset = dataset
        self.batch_size = batch_size
        self.num_replicas = num_replicas
        self.rank = rank
        self.epoch = 0
        self.drop_last = drop_last

        # If the dataset length is evenly divisible by # of replicas, then there
        # is no need to drop any data, since the dataset will be split equally.
        if self.drop_last and len(self.dataset) % self.num_replicas != 0:
            # Split to nearest available length that is evenly divisible.
            # This is to ensure each rank receives the same amount of data when
            # using this Sampler.
            self.num_samples = math.ceil((len(self.dataset) - self.num_replicas) / self.num_replicas)
        else:
            self.num_samples = math.ceil(len(self.dataset) / self.num_replicas)
        self.total_size = self.num_samples * self.num_replicas
        self.seed = seed

        if lengths is None:
            if not isinstance(dataset[0], dict) or "input_ids" not in dataset[0]:
                raise ValueError(
                    "Can only automatically infer lengths for datasets whose items are dictionaries with an "
                    "'input_ids' key."
                )
            lengths = [len(feature["input_ids"]) for feature in dataset]
        self.lengths = lengths

    def __iter__(self) -> Iterator:
        # Deterministically shuffle based on epoch and seed
        g = torch.Generator()
        g.manual_seed(self.seed + self.epoch)
        indices = get_indices(self.lengths, self.batch_size, generator=g)

        if not self.drop_last:
            # add extra samples to make it evenly divisible
            indices += indices[: (self.total_size - len(indices))]
        else:
            # remove tail of data to make it evenly divisible.
            indices = indices[: self.total_size]
        assert len(indices) == self.total_size

        # subsample
        indices = indices[self.rank : self.total_size : self.num_replicas]
        assert len(indices) == self.num_samples

        return iter(indices)

In [ ]:
# https://huggingface.co/transformers/master/_modules/transformers/trainer.html#Trainer.get_train_dataloader

from typing import Optional
from transformers.training_args import ParallelMode
import torch
class CustomTrainer(Trainer):
    def get_processes_info(self):
        # Gather the number of processes and this process index.
        if self.args.parallel_mode == ParallelMode.TPU:
            num_processes = xm.xrt_world_size()
            process_index = xm.get_ordinal()
        elif (
            self.args.parallel_mode == ParallelMode.DISTRIBUTED
        ):
            num_processes = dist.get_world_size()
            process_index = dist.get_rank()
        else:
            num_processes = 1
            process_index = 0

        return num_processes, process_index

    def get_train_dataloader(self) -> DataLoader:
        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")

        num_processes, process_index = self.get_processes_info()

        train_sampler = DistributedLengthSortedSampler(
            self.train_dataset, 
            self.args.train_batch_size, 
            num_replicas=num_processes, 
            rank=process_index
        )

        return DataLoader(
            self.train_dataset,
            batch_size=self.args.train_batch_size,
            sampler=train_sampler,
            shuffle=False,
            collate_fn=self.data_collator,
            drop_last=self.args.dataloader_drop_last,
            num_workers=self.args.dataloader_num_workers,
        )

    def get_eval_dataloader(self, eval_dataset: Optional[Dataset] = None) -> DataLoader:
        if eval_dataset is None and self.eval_dataset is None:
            raise ValueError("Trainer: evaluation requires an eval_dataset.")
        elif eval_dataset is not None and not isinstance(eval_dataset, collections.abc.Sized):
            raise ValueError("eval_dataset must implement __len__")
        elif isinstance(eval_dataset, Dataset):
            self._remove_unused_columns(eval_dataset, description="evaluation")
        eval_dataset = eval_dataset if eval_dataset is not None else self.eval_dataset

        num_processes, process_index = self.get_processes_info()

        eval_sampler = DistributedLengthSortedSampler(
            eval_dataset, 
            self.args.eval_batch_size, 
            num_replicas=num_processes, 
            rank=process_index
        )

        return DataLoader(
            eval_dataset,
            sampler=eval_sampler,
            batch_size=self.args.eval_batch_size,
            collate_fn=self.data_collator,
            drop_last=self.args.dataloader_drop_last,
            num_workers=self.args.dataloader_num_workers,
            pin_memory=self.args.dataloader_pin_memory,
        )

#### обучение1

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy = "steps",   
    num_train_epochs=3,              # total number of training epochs
    learning_rate=0.00005,     #5e-5,  0.00005
    overwrite_output_dir=True, 
    logging_steps = 1000,
    save_steps = 9000,
    weight_decay=0.01,               # strength of weight decay
    warmup_steps=3000,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
)

In [ ]:
cTrainer = CustomTrainer(model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_eval,
    data_collator=data_collator,
)

In [ ]:
cTrainer.train()

Step,Training Loss,Validation Loss,Runtime,Samples Per Second
1000,11.305600,10.797035,20.574000,1834.066000
2000,10.658100,10.528041,20.554000,1835.848000
3000,10.441400,10.333051,20.568900,1834.520000
4000,10.263500,10.141434,20.561500,1835.178000
5000,10.099300,10.007213,20.577100,1833.784000
6000,10.003700,9.938433,20.599200,1831.819000
7000,9.925300,9.861758,20.571300,1834.299000
8000,9.898600,9.777792,20.576300,1833.860000
9000,9.783100,9.717603,20.571400,1834.291000
10000,9.743300,9.701392,20.565900,1834.786000


TrainOutput(global_step=16089, training_loss=9.999799261899872, metrics={'train_runtime': 1419.7196, 'train_samples_per_second': 11.333, 'total_flos': 9151684899030360, 'epoch': 3.0})

In [ ]:
# cTrainer.save_model('/content/drive/MyDrive/chatbot_project/models/tuned0')

In [ ]:
predict("Вася покачал головой", 3)

['[CLS]', 'вас', '##я', 'пока', '##чал', 'голово', '##и', '[SEP]']
Pos = 3

Index = 852
Value = 6.0784010887146
Token = ##и
 [CLS] васяичал головои [SEP]

Index = 851
Value = 4.550236225128174
Token = и
 [CLS] вася ичал головои [SEP]

Index = 845
Value = 4.23728084564209
Token = в
 [CLS] вася вчал головои [SEP]

Index = 869
Value = 3.987553119659424
Token = с
 [CLS] вася счал головои [SEP]

Index = 1516
Value = 3.4218595027923584
Token = по
 [CLS] вася почал головои [SEP]



### разморозка

In [ ]:
for param in model.base_model.parameters():
     param.requires_grad = True

#### обучение2

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy = "steps",   
    num_train_epochs=1,              # total number of training epochs
    learning_rate=0.00005,
    overwrite_output_dir=True, 
    logging_steps = 1000,
    save_steps = 9000,
    weight_decay=0.01,               # strength of weight decay
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
)
cTrainer = CustomTrainer(model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_eval,
    data_collator=data_collator,
)

In [ ]:
cTrainer.train()

Step,Training Loss,Validation Loss,Runtime,Samples Per Second
1000,4.539700,4.383358,85.526500,520.692000
2000,4.449500,4.304394,85.748200,519.346000
3000,4.344300,4.216086,85.619800,520.125000
4000,4.265000,4.155223,85.646100,519.965000
5000,4.193600,4.107662,85.679600,519.762000
6000,4.183100,4.069064,85.756800,519.294000


TrainOutput(global_step=6199, training_loss=4.322434404585165, metrics={'train_runtime': 2891.4315, 'train_samples_per_second': 2.144, 'total_flos': 5362539318092574, 'epoch': 1.0})

In [ ]:
cTrainer.train()

Step,Training Loss,Validation Loss,Runtime,Samples Per Second
1000,7.708500,7.069294,20.602600,1831.513000
2000,6.748700,6.407361,20.601200,1831.642000
3000,6.237100,6.081542,20.608100,1831.030000
4000,5.934900,5.694358,20.616600,1830.272000
5000,5.650800,5.472235,20.598300,1831.902000
6000,5.359100,5.331687,20.613800,1830.524000
7000,5.208300,5.139632,20.608600,1830.987000
8000,5.103800,4.967333,20.608700,1830.974000
9000,4.915700,4.881063,20.621800,1829.815000
10000,4.834300,4.829659,20.626000,1829.440000


TrainOutput(global_step=37541, training_loss=4.454563163935555, metrics={'train_runtime': 6837.7303, 'train_samples_per_second': 5.49, 'total_flos': 21353931431070840, 'epoch': 7.0})

In [ ]:
cTrainer.train()

Step,Training Loss,Validation Loss,Runtime,Samples Per Second
1000,7.752700,8.703333,33.065300,1155.017000
2000,6.743100,7.438605,33.057500,1155.290000
3000,6.319500,7.193581,33.056600,1155.321000
4000,5.976200,7.203521,33.049900,1155.557000
5000,5.679000,6.745315,33.087200,1154.252000
6000,5.460900,5.876411,33.059700,1155.213000
7000,5.185100,6.115569,33.049700,1155.563000
8000,5.041800,6.003054,33.050300,1155.540000
9000,4.919800,5.810862,33.063300,1155.086000
10000,4.810400,5.861529,33.058700,1155.248000


TrainOutput(global_step=26780, training_loss=4.793068521284541, metrics={'train_runtime': 5123.1253, 'train_samples_per_second': 5.227, 'total_flos': 15230404732095360, 'epoch': 5.0})

In [ ]:
cTrainer.evaluate()

{'epoch': 7.0,
 'eval_loss': 3.7531611919403076,
 'eval_runtime': 20.6445,
 'eval_samples_per_second': 1827.795}

In [ ]:
# cTrainer.save_model('/content/drive/MyDrive/chatbot_project/models/tuned0')

### проверки (для дебага)

In [ ]:
from torch.utils.data.dataloader import DataLoader
loader = cTrainer.get_train_dataloader()

In [ ]:
token_stats = {}
bad = 0
all = 0
for batch in loader:
    l = batch['input_ids'].size()[1]
    b_size = batch['input_ids'].size()[0]
    is_bad = False
    for pos in range(b_size):
        tokens = tokenizer.convert_ids_to_tokens(batch['input_ids'].tolist()[pos])
        for token in tokens:
            if (token == '[PAD]'):
                is_bad = True
            if (token_stats.get(token) == None):
                token_stats[token] = 0
            token_stats[token] += 1
    all += 1
    if is_bad:
        bad += 1

In [ ]:
print(all, bad)

5363 14


In [ ]:
sort_stats = [[k, v] for k, v in sorted(token_stats.items(), key=lambda item: -item[1])]
sort_stats[:25]

[['[CLS]', 343226],
 ['[SEP]', 343225],
 ['[MASK]', 260727],
 ['ты', 46188],
 ['.', 39754],
 ['##и', 39262],
 ['не', 28790],
 ['в', 28408],
 ['и', 23700],
 [',', 22971],
 ['?', 20600],
 ['есть', 19684],
 ['на', 19553],
 ['я', 16174],
 ['##а', 11384],
 ['##ка', 9017],
 ['а', 8893],
 ['-', 8478],
 ['с', 8359],
 ['##ы', 8180],
 ['_', 7358],
 ['##ои', 7225],
 ['что', 7007],
 ['##ся', 6997],
 ['но', 6843]]

In [ ]:
stats = {}
for i in range(1, 210):
  stats[i] = 0

In [ ]:
ind = 0
for batch in loader:
    l = batch['input_ids'].size()[1]
    stats[l] += 1
    if np.random.rand(1) < 0.00001:
        print("--- batch begin ---")
        b_size = batch['input_ids'].size()[0]
        for pos in range(b_size):
            print(ind, l)
            print(tokenizer.convert_ids_to_tokens(batch['input_ids'].tolist()[pos]))
            print(tokenizer.convert_ids_to_tokens(batch['labels'].tolist()[pos]))
            print()
            ind += 1
        print("--- batch end ---")

In [ ]:
stats

{1: 0,
 2: 0,
 3: 44,
 4: 56,
 5: 328,
 6: 613,
 7: 743,
 8: 670,
 9: 663,
 10: 546,
 11: 405,
 12: 260,
 13: 172,
 14: 129,
 15: 106,
 16: 98,
 17: 92,
 18: 91,
 19: 84,
 20: 83,
 21: 76,
 22: 69,
 23: 63,
 24: 56,
 25: 53,
 26: 47,
 27: 45,
 28: 43,
 29: 42,
 30: 38,
 31: 34,
 32: 34,
 33: 32,
 34: 29,
 35: 30,
 36: 29,
 37: 25,
 38: 25,
 39: 22,
 40: 23,
 41: 21,
 42: 19,
 43: 18,
 44: 17,
 45: 18,
 46: 18,
 47: 14,
 48: 14,
 49: 14,
 50: 13,
 51: 11,
 52: 13,
 53: 11,
 54: 11,
 55: 9,
 56: 10,
 57: 8,
 58: 8,
 59: 8,
 60: 8,
 61: 7,
 62: 8,
 63: 6,
 64: 6,
 65: 5,
 66: 6,
 67: 5,
 68: 5,
 69: 4,
 70: 5,
 71: 4,
 72: 4,
 73: 4,
 74: 4,
 75: 3,
 76: 3,
 77: 4,
 78: 3,
 79: 2,
 80: 3,
 81: 2,
 82: 3,
 83: 2,
 84: 2,
 85: 2,
 86: 2,
 87: 2,
 88: 1,
 89: 2,
 90: 2,
 91: 1,
 92: 1,
 93: 1,
 94: 1,
 95: 2,
 96: 1,
 97: 1,
 98: 1,
 99: 2,
 100: 1,
 101: 1,
 102: 1,
 103: 0,
 104: 1,
 105: 1,
 106: 0,
 107: 1,
 108: 1,
 109: 0,
 110: 1,
 111: 1,
 112: 0,
 113: 1,
 114: 0,
 115: 1,
 116: 0,


In [ ]:
stats

{1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 34,
 12: 764,
 13: 1939,
 14: 1558,
 15: 735,
 16: 245,
 17: 58,
 18: 24,
 19: 3,
 20: 2,
 21: 0,
 22: 0,
 23: 0,
 24: 0,
 25: 0,
 26: 0,
 27: 0,
 28: 0,
 29: 0,
 30: 0,
 31: 0,
 32: 0,
 33: 0,
 34: 0,
 35: 0,
 36: 1,
 37: 0,
 38: 0,
 39: 0,
 40: 0,
 41: 0,
 42: 0,
 43: 0,
 44: 0,
 45: 0,
 46: 0,
 47: 0,
 48: 0,
 49: 0}

In [ ]:
ind = 0
for elem in loader:
    if np.random.rand(1) < 0.0001:
        print(ind)
        l = len(elem['input_ids'].tolist()[0])
        print(tokenizer.convert_ids_to_tokens(elem['input_ids'].tolist()[0]))
        print(tokenizer.convert_ids_to_tokens(elem['labels'].tolist()[0]))
        elem.to(cuda)
        outputs = model(**elem)
        print("loss =", outputs.loss.item())
        pred = outputs.logits
        print(pred.size())
        for ind in range(l):
            print("word number", ind)
            for num_best in range(3):
                pred_sort = torch.argsort(pred[0, ind])
                best = pred_sort[-1-num_best]
                token = tokenizer.convert_ids_to_tokens([best])[0]
                print(pred[0, ind, best].item(), token)
        print()
    ind += 1

1248
['[CLS]', 'мак', '##ле', '##од', 'успел', 'ух', '##вати', '##ть', '[SEP]']
['[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '[UNK]', '##вати', '[UNK]', '[UNK]']
loss = 9.81553840637207
torch.Size([8, 9, 119547])
word number 0
2.827181339263916 в
2.559532403945923 с
1.752619743347168 по
word number 1
5.105445861816406 с
4.560785293579102 в
3.836024761199951 за
word number 2
4.479654312133789 в
4.025650501251221 с
3.282163143157959 я
word number 3
4.103397846221924 в
3.868391275405884 с
3.0620367527008057 за
word number 4
3.578986167907715 за
3.3578600883483887 в
2.7325286865234375 о
word number 5
2.9740023612976074 за
2.7133827209472656 тем
2.4391937255859375 этого
word number 6
3.463263511657715 за
3.3294191360473633 тем
3.2184057235717773 этого
word number 7
2.9255802631378174 за
2.216721534729004 ##а
2.145681858062744 помимо
word number 8
2.562978506088257 за
2.4669344425201416 в
1.9445308446884155 с

5044
['[CLS]', '[MASK]', '##ски', '##и', 'полк', '[SEP]']
['[UNK]', 'волын', '[UN

In [ ]:
res = tokenized_dataset_eval[10]
tokenizer.convert_ids_to_tokens(res['input_ids'])

['[CLS]', 'дже', '##ими', 'попросил', 'уделить', '[SEP]']

In [ ]:
import torch
cuda = torch.device('cuda:0')

In [ ]:
tokenizer.tokenize("Вася кивнул головой")

['вас', '##я', 'ки', '##в', '##нул', 'голово', '##и']

In [ ]:
inputs = tokenizer("Вася покачал головой", return_tensors="pt").to(cuda)
outputs = model.forward(**inputs, output_hidden_states=True)
outputs.logits.size()

torch.Size([1, 8, 119547])

In [ ]:
pos = 7
predictions = outputs.logits
predicted_index = torch.argsort(predictions[0, pos])
for i in range(10):
    ind = predicted_index[-1-i]
    predicted_token = tokenizer.convert_ids_to_tokens([ind])[0]
    print(ind.item())
    print(predictions[0, pos, ind].item(), predicted_token)

845
-88.8191909790039 в
1469
-88.86975860595703 на
1703
-89.49671936035156 из
1641
-89.55485534667969 от
3468
-89.68830871582031 после
869
-89.68840789794922 с
2748
-89.74311828613281 для
875
-89.74755096435547 у
1758
-89.76760864257812 за
612
-89.89922332763672 о


## Сохранение

In [ ]:
# сохранение на выделенной машине
cTrainer.save_model('./saved_model_tuned')

In [ ]:
# сохранение на гугл диске
# cTrainer.save_model('/content/drive/MyDrive/chatbot_project/models/tuned1_simple')

In [ ]:
model_end = BertModel.from_pretrained('./saved_model_tuned')

Some weights of BertModel were not initialized from the model checkpoint at ./saved_model_tuned and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Проверка

In [ ]:
# model = model_end

### угадывание слов

In [ ]:
predict("Вася покачал головой и [MASK]", 8)

['[CLS]', 'вас', '##я', 'пока', '##чал', 'голово', '##и', 'и', '[MASK]', '[SEP]']
Pos = 8

Index = 132
Value = 5.023471355438232
Token = .
 [CLS] вася покачал головои и . [SEP]

Index = 33410
Value = 4.5508294105529785
Token = встал
 [CLS] вася покачал головои и встал [SEP]

Index = 128
Value = 4.336976051330566
Token = ,
 [CLS] вася покачал головои и , [SEP]

Index = 39846
Value = 4.303760051727295
Token = остановился
 [CLS] вася покачал головои и остановился [SEP]

Index = 26654
Value = 4.060601234436035
Token = поднялся
 [CLS] вася покачал головои и поднялся [SEP]



In [ ]:
predict("Наконец-то ящерица пошевелилась.", 4)

['[CLS]', 'наконец', '-', 'то', 'ящерица', 'пош', '##еве', '##ли', '##лась', '.', '[SEP]']
Pos = 4

Index = 1699
Value = 6.0463786125183105
Token = не
 [CLS] наконец - то не пошевелилась . [SEP]

Index = 28952
Value = 5.701249599456787
Token = голова
 [CLS] наконец - то голова пошевелилась . [SEP]

Index = 877
Value = 5.089743137359619
Token = я
 [CLS] наконец - то я пошевелилась . [SEP]

Index = 878
Value = 4.593038082122803
Token = ##я
 [CLS] наконец - тоя пошевелилась . [SEP]

Index = 23725
Value = 4.467600345611572
Token = опять
 [CLS] наконец - то опять пошевелилась . [SEP]



In [ ]:
predict("Он посмотрел под холодильник", 4)

['[CLS]', 'он', 'посмотрел', 'под', 'холодильник', '[SEP]']
Pos = 4

Index = 114485
Value = 9.993294715881348
Token = холодильник
 [CLS] он посмотрел под холодильник [SEP]

Index = 111578
Value = 6.426549911499023
Token = диван
 [CLS] он посмотрел под диван [SEP]

Index = 74989
Value = 5.867487907409668
Token = шкаф
 [CLS] он посмотрел под шкаф [SEP]

Index = 111871
Value = 5.091196537017822
Token = хороши
 [CLS] он посмотрел под хороши [SEP]

Index = 851
Value = 4.8505096435546875
Token = и
 [CLS] он посмотрел под и [SEP]



In [ ]:
predict("Николай [MASK] в магазин", 3)

['[CLS]', 'никол', '##аи', '[MASK]', 'в', 'магазин', '[SEP]']
Pos = 3

Index = 26901
Value = 8.24205493927002
Token = вошел
 [CLS] николаи вошел в магазин [SEP]

Index = 40794
Value = 7.135937213897705
Token = пошел
 [CLS] николаи пошел в магазин [SEP]

Index = 24651
Value = 6.9975385665893555
Token = пришел
 [CLS] николаи пришел в магазин [SEP]

Index = 11346
Value = 6.926224231719971
Token = вернулся
 [CLS] николаи вернулся в магазин [SEP]

Index = 16505
Value = 6.588040828704834
Token = отправился
 [CLS] николаи отправился в магазин [SEP]



In [ ]:
predict("Вчера ты [MASK] дома.", 3)

['[CLS]', 'вчера', 'ты', '[MASK]', 'дома', '.', '[SEP]']
Pos = 3

Index = 2067
Value = 6.697117328643799
Token = был
 [CLS] вчера ты был дома . [SEP]

Index = 3370
Value = 6.029049873352051
Token = была
 [CLS] вчера ты была дома . [SEP]

Index = 16311
Value = 5.569386005401611
Token = остался
 [CLS] вчера ты остался дома . [SEP]

Index = 23532
Value = 5.548834323883057
Token = видел
 [CLS] вчера ты видел дома . [SEP]

Index = 107835
Value = 5.067227363586426
Token = будешь
 [CLS] вчера ты будешь дома . [SEP]



### близость по смыслу

In [ ]:
"""
Класс, который берёт слово и выдаёт эмбед этого слова.
Нужен для check_similarity
"""
class EmbedModel:
    def __init__(self, word_to_token, token_to_embed):
        self.word_to_token = word_to_token
        self.token_to_embed = token_to_embed

    def __call__(self, word):
        tokens = torch.Tensor(self.word_to_token.encode(word, add_special_tokens=False)).long().to(cuda)
        out = self.token_to_embed(tokens)
        return out.mean(dim=0)

baseline_model = EmbedModel(tokenizer, model_begin.embeddings.word_embeddings.to(cuda))
new_model = EmbedModel(tokenizer, model.bert.embeddings.word_embeddings)

"""
Обрабатывает тест.
@parr
"""
def check_similarity(model, similar_word1, similar_word2, different_word, silent=False):
    v1, v2, v3 = model(similar_word1), model(similar_word2), model(different_word)
    d12 = float(nn.CosineSimilarity(dim=0)(v1, v2))
    d13 = float(nn.CosineSimilarity(dim=0)(v1, v3))
    d23 = float(nn.CosineSimilarity(dim=0)(v2, v3))
    if not silent:
        print(f'Distance between similar words: {d12:.4f}')
        print(f'Distance between similar words: {d13:.4f}')
        print(f'Distance between similar words: {d23:.4f}')
    return (d12 > d13) and (d12 > d23)

print(check_similarity(baseline_model, "вошёл", "зашёл", "красивый"))
print(check_similarity(new_model, "вошёл", "пришёл", "красивый"))

Distance between similar words: 0.5769
Distance between similar words: 0.5771
Distance between similar words: 0.5649
False
Distance between similar words: 0.6201
Distance between similar words: 0.5403
Distance between similar words: 0.5358
True


In [ ]:
# первыми двумя - похожие слова, третьим - лишнее
words = ['вошёл', 'пришёл', 'красивый']

print(f'Старая модель: {check_similarity(baseline_model, *words)}')
print(f'Новая модель: {check_similarity(new_model, *words)}')

Distance between similar words: 0.5450
Distance between similar words: 0.5771
Distance between similar words: 0.5834
Старая модель: False
Distance between similar words: 0.6201
Distance between similar words: 0.5403
Distance between similar words: 0.5358
Новая модель: True


In [ ]:
def test_word(model1, model2, words):
  print(words)
  print(f'Старая модель: {check_similarity(model1, *words)}')
  print(f'Новая модель: {check_similarity(model2, *words)}')
  print()

In [ ]:
test_word(baseline_model, new_model, ['вошёл', 'вышел', 'стул'])
test_word(baseline_model, new_model, [ 'стул', "стол", "стоять"])
test_word(baseline_model, new_model, ['синий', "красный", "и"])
test_word(baseline_model, new_model, ['он', "ты", "новый"])
test_word(baseline_model, new_model, ['он', "они", "тоже"])
test_word(baseline_model, new_model, ['зашёл', 'пришёл', 'красивый'])
test_word(baseline_model, new_model, ['ты', 'я', 'свинья'])
test_word(baseline_model, new_model, ['быстрый', 'сложная', 'день'])

['вошёл', 'вышел', 'стул']
Distance between similar words: 0.5845
Distance between similar words: 0.5582
Distance between similar words: 0.5755
Старая модель: True
Distance between similar words: 0.6320
Distance between similar words: 0.5837
Distance between similar words: 0.5785
Новая модель: True

['стул', 'стол', 'стоять']
Distance between similar words: 0.4566
Distance between similar words: 0.5479
Distance between similar words: 0.4077
Старая модель: False
Distance between similar words: 0.5463
Distance between similar words: 0.4884
Distance between similar words: 0.3453
Новая модель: True

['синий', 'красный', 'и']
Distance between similar words: 0.6387
Distance between similar words: 0.5074
Distance between similar words: 0.4853
Старая модель: True
Distance between similar words: 0.6047
Distance between similar words: 0.2867
Distance between similar words: 0.4277
Новая модель: True

['он', 'ты', 'новый']
Distance between similar words: 0.4701
Distance between similar words: 0.50